In [1]:
%load_ext autoreload
import pandas as pd
import numpy as np

# Loading the Data

In [2]:
X_train_1 = pd.read_csv('data/dengue_features_train.csv')
y_train = pd.read_csv('data/dengue_labels_train.csv')['total_cases']
attr = list(X_train_1)
attr

['city',
 'year',
 'weekofyear',
 'week_start_date',
 'ndvi_ne',
 'ndvi_nw',
 'ndvi_se',
 'ndvi_sw',
 'precipitation_amt_mm',
 'reanalysis_air_temp_k',
 'reanalysis_avg_temp_k',
 'reanalysis_dew_point_temp_k',
 'reanalysis_max_air_temp_k',
 'reanalysis_min_air_temp_k',
 'reanalysis_precip_amt_kg_per_m2',
 'reanalysis_relative_humidity_percent',
 'reanalysis_sat_precip_amt_mm',
 'reanalysis_specific_humidity_g_per_kg',
 'reanalysis_tdtr_k',
 'station_avg_temp_c',
 'station_diur_temp_rng_c',
 'station_max_temp_c',
 'station_min_temp_c',
 'station_precip_mm']

## Cleaning the noisy training data

In [3]:
def bools_to_indexes(booleans):
    r = []
    for idx, x in enumerate(booleans):
        if x:
            r.append(idx)
    return r

idx = bools_to_indexes(X_train_1['weekofyear'] == 53)
y_train.drop(idx, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_train_1.drop(idx, inplace=True)
X_train_1.reset_index(drop=True, inplace=True)
X_train_1.shape

(1451, 24)

# Model Selection

In [77]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
score_metric='neg_mean_absolute_error'
jobs=-1 #-1 to make it execute in parallel
verbose_level = 0
random_n = 42
base_args = {'estimator': None, 'param_distributions': None, 'n_iter': None, 'scoring': score_metric, 'n_jobs': jobs, 'cv': None, 'verbose': verbose_level, 'random_state': random_n, 'return_train_score': True, 'iid': True}

## SVR
* The results with the kernel *sigmoid* and *poly* were too bad, so we removed them.

In [78]:
k_folds=10
n_iter_search = 20
C = sp_randint(0, 10000)
params = {'kernel':['linear'], 'gamma':['scale'], 'C': C}
SVR_optimizer = RandomizedSearchCV(estimator=SVR(), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

## Regression Trees
* 18.01 - with 2 previous weeks & without PCA & with (max_depth=6, min_samples_leaf=0.1611807565247405, min_samples_split=0.11193019906931466)
* 18.29 - With PCA at 0.9
* 18.27 - With PCA at 0.95
* 18.36 - With PCA at 0.65. PCA appears to be only making the model worse.
* 18.38 - Without PCA and with previous weeks. Clearly the previous weeks are useful
* 17.87 - Without PCA and with 3 previous weeks
* 17.86 - Without PCA and with 4 previous weeks
* 18.28 - With PCA 0.95 and 3 previous weeks fixed
* 9.16 - Without PCA, with 3 weeks and 1 last infection (max_depth=5, min_samples_leaf=0.03, min_samples_split=0.108)
* **9.04** - Without PCA, with 3 weeks and 1 last infection (max_depth=5, min_samples_leaf=0.03, min_samples_split=0.108)

In [6]:
k_folds=10
n_iter_search = 100
min_samples = sp_uniform(0.01, 0.35)
params = {'criterion':['mae'], 'max_depth': sp_randint(2, 10), 'min_samples_split': min_samples, 'min_samples_leaf': min_samples}
Tree_optimizer = RandomizedSearchCV(estimator=DecisionTreeRegressor(), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

## Random Forests
* 18.34 With 4 previous weeks and without PCA
* 17.79 With fixed 3 previous weeks and PCA at 0.95 (n_estimators= ?, max_depth = 2, min_samples_leaf=0.112, min_samples_split=0.224)
* 17.74 With fixed 3 previous weeks and without PCA (n_estimators= 13 max_depth = 5, min_samples_leaf=0.09, min_samples_split=0.24)
* **9.13** with 3 previous weeks and 1 last infected (n_estimators=9 max_depth = 9, min_samples_leaf=0.014, min_samples_split=0.07)
* 9.22 with 3 previous weeks and 3 last infected (n_estimators=9 max_depth = 9, min_samples_leaf=0.014, min_samples_split=0.08)

In [7]:
k_folds=10
n_iter_search = 40
params = {'n_estimators': sp_randint(2,50), 'criterion':['mae'], 'max_depth': sp_randint(2, 10)}
Forest_optimizer = RandomizedSearchCV(estimator=RandomForestRegressor(n_jobs=-1), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

## Adaboost of Trees
* 10.78 - With 3 last weeks a 3 last infected 
* **8.49** - With 3 last weeks a 3 last infected and only max_depth tuned.

In [8]:
k_folds=10
n_iter_search = 20
params = {'base_estimator__criterion':['mae'], 'base_estimator__max_depth': sp_randint(2,7), 'base_estimator__n_estimators': sp_randint(40, 100)}
AdaTree_optimizer = RandomizedSearchCV(estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor()), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

## KNN
* 21.349 - with PCA at 0.65 & 2 previous weeks
* 20.36  - without PCA

In [9]:
k_folds=10
n_iter_search = 100
params = {'n_neighbors': sp_randint(3,150), 'weights': ['uniform', 'distance']}
KNN_optimizer = RandomizedSearchCV(estimator=KNeighborsRegressor(n_jobs=-1), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

# Optimization

In [76]:
%autoreload
from OurPipeline import create_pipeline

#pipeline = create_pipeline(attr, estimator_optimizer=RandomizedSearchCV(None, None), n_weeks=0, n_weeks_infected=0)
optimizers=[Tree_optimizer]#, Forest_optimizer, AdaTree_optimizer, KNN_optimizer, SVR_optimizer]
weeks = [1,2]
weeks_infected = [3,4]
pca = [PCA(0.95), None]

n_total = len(optimizers) * len(weeks) * len(weeks_infected) * len(pca)

results=[]
best_attempt = None
best_score = np.inf
idx=0
for opt in optimizers:
    for w in weeks:
        for wi in weeks_infected:
            for p in pca:
                pipeline = create_pipeline(attr, n_weeks=w, n_weeks_infected=wi, estimator_optimizer=opt, pca=p)
                pipeline.fit(X_train_1, y_train)
                score = pipeline.named_steps['est_opt'].best_score_
                best_estimator = pipeline.named_steps['est_opt'].best_estimator_
                attempt = [best_estimator, w, wi, p, score]
                if abs(score) < best_score:
                    best_score = abs(score)
                    best_estimator = attempt
                    print('\nBest score of {} with the estimator {}'.format(best_score, best_estimator))
                idx+=1
                print(str(idx) + '/' + str(n_total), end='\t')
                results.append(attempt)
                print(results)
                    

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 239 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 739 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done 993 out of 1000 | elapsed:   50.0s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   50.1s finished



Best score of 12.7829083390765 with the estimator [DecisionTreeRegressor(criterion='mae', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.048518173584686866,
           min_samples_split=0.08977730688967958,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'), 1, 3, PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False), -12.7829083390765]
1/8	[[DecisionTreeRegressor(criterion='mae', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.048518173584686866,
           min_samples_split=0.08977730688967958,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'), 1, 3, PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=No

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   41.2s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  1.6min finished



Best score of 8.982081323225362 with the estimator [DecisionTreeRegressor(criterion='mae', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.02801756293749627,
           min_samples_split=0.107526262482814,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'), 1, 3, None, -8.982081323225362]
2/8	[[DecisionTreeRegressor(criterion='mae', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.048518173584686866,
           min_samples_split=0.08977730688967958,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'), 1, 3, PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False), -12.7829083390765], [DecisionTreeRegressor(criterion='mae', max_dep

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 420 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 920 tasks      | elapsed:   45.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   49.9s finished


3/8	[[DecisionTreeRegressor(criterion='mae', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.048518173584686866,
           min_samples_split=0.08977730688967958,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'), 1, 3, PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False), -12.7829083390765], [DecisionTreeRegressor(criterion='mae', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.02801756293749627,
           min_samples_split=0.107526262482814,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'), 1, 3, None, -8.982081323225362], [DecisionTreeRegressor(criterion='mae', max_depth=7, max_features=None,
           max_leaf_nodes=

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.1s


KeyboardInterrupt: 

In [ ]:
%autoreload
from OurPipeline import create_pipeline

pipeline = create_pipeline(attr, n_weeks=2, n_weeks_infected=2, estimator_optimizer=Tree_optimizer, pca=None)

In [69]:
pipeline.fit(X_train_1, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   56.8s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  2.2min finished


Pipeline(memory=None,
     steps=[('imputer', ContinuityImputer(attributes=['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k', 'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 'reanalysis_precip_amt_kg_per_m2', 'reanalys...t=True,
          return_train_score=True, scoring='neg_mean_absolute_error',
          verbose=1))])

In [70]:
pipeline.named_steps['est_opt'].best_score_

-9.066505858028945

In [72]:
2.copy()

SyntaxError: invalid syntax (<ipython-input-72-0452fe5c3e92>, line 1)